In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [91]:
class TetMesh:
    def __init__(self,origin,nsteps,step_vector):
        self.origin = np.array(origin) 
        self.step_vector = np.array(step_vector)     
        self.nsteps = np.array(nsteps)
        self.nsteps_cells = self.nsteps - 1
        self.n_cell_x = self.nsteps[0] - 1
        self.n_cell_y = self.nsteps[1] - 1
        self.n_cell_z = self.nsteps[2] - 1
        
    def get_tetra(self,pos):
        pos = np.array(pos)
        # find which cell the points are in
        gi, inside = self.position_to_cell_corners(pos)
        # now determine the tetra from the corners
        tetra = np.dstack([gi[:,[0,4,5,3]],
                        gi[:,[4,6,7,5]],
                        gi[:,[0,1,6,4]],
                        gi[:,[0,6,5,4]],
                        gi[:,[0,6,5,2]]
                          ])
        xi,yi,zi = self.global_index_to_node_index(tetra)
        apoints = self.node_indexes_to_position(xi,yi,zi)
        cc = []
#         points = points.swapaxes(0,3)
#         points = points.swapaxes(0,1)
        for ii in range(apoints.shape[1]):
            points = apoints[:,ii,:,:].T
            vap = pos[ii,None,None,:] - points[None,:, 0,:]
            vbp = pos[ii,None,None,:] - points[None,:, 1, :]
            #         # vcp = p - points[:, 2, :]
            #         # vdp = p - points[:, 3, :]
            vab = points[None,:,  1, :] - points[None, :, 0, :]
            vac = points[None,:,  2, :] - points[None, :, 0, :]
            vad = points[None,:,  3, :] - points[None, :, 0, :]
            vbc = points[None,:,  2, :] - points[None, :, 1, :]
            vbd = points[None,:,  3, :] - points[None, :, 1, :]

            va = np.einsum('ikj, ikj->ik', vbp, np.cross(vbd, vbc, axisa=2, axisb=2)) / 6.
            vb = np.einsum('ikj, ikj->ik', vap, np.cross(vac, vad, axisa=2, axisb=2)) / 6.
            vc = np.einsum('ikj, ikj->ik', vap, np.cross(vad, vab, axisa=2, axisb=2)) / 6.
            vd = np.einsum('ikj, ikj->ik', vap, np.cross(vab, vac, axisa=2, axisb=2)) / 6.

            v = np.einsum('ikj, ikj->ik', vab, np.cross(vac, vad, axisa=2, axisb=2)) / 6.
            c = np.zeros((va.shape[0],va.shape[1],4))

            #print(va.shape)
            c[:,:,0] = va / v
            c[:,:,1]= vb / v
            c[:,:,2] = vc / v
            c[:,:,3] = vd / v
            cc.append(c)
        return cc
    def calc_bary_centre(self,points,position):

        npts = len(e)
        vap = position - points[:, 0, :]
        vbp = position - points[:, 1, :]
        # vcp = p - points[:, 2, :]
        # vdp = p - points[:, 3, :]
        vab = points[:, 1, :] - points[:, 0, :]
        vac = points[:, 2, :] - points[:, 0, :]
        vad = points[:, 3, :] - points[:, 0, :]
        vbc = points[:, 2, :] - points[:, 1, :]
        vbd = points[:, 3, :] - points[:, 1, :]
        vbp * np.cross(vbd, vbc, axisa=1, axisb=1)
        va = np.sum(vbp * np.cross(vbd, vbc, axisa=1, axisb=1), axis=1) / 6.
        vb = np.sum(vap * np.cross(vac, vad, axisa=1, axisb=1), axis=1) / 6.
        vc = np.sum(vap * np.cross(vad, vab, axisa=1, axisb=1), axis=1) / 6.
        vd = np.sum(vap * np.cross(vab, vac, axisa=1, axisb=1), axis=1) / 6.
        v = np.sum(vab * np.cross(vac, vad, axisa=1, axisb=1), axis=1) / 6.
        c = np.zeros((4, npts))
        c[0, :] = va / v
        c[1, :] = vb / v
        c[2, :] = vc / v
        c[3, :] = vd / v
        return c
    
    
    def inside(self, pos):

        # check whether point is inside box
        inside = np.ones(pos.shape[0]).astype(bool)
        for i in range(3):
            inside *= pos[:, i] > self.origin[None, i]
            inside *= pos[:, i] < self.origin[None, i] + \
                      self.step_vector[None, i] * self.nsteps_cells[None, i]
        return inside    
    def global_indicies(self,indexes):
        indexes = np.array(indexes).swapaxes(0, 2)
        return indexes[:, :, 0] + self.nsteps[None, None, 0] * indexes[:, :,
                                                               1] + \
               self.nsteps[None, None, 0] * self.nsteps[
                   None, None, 1] * indexes[:, :, 2]
    def cell_corner_indexes(self, x_cell_index, y_cell_index, z_cell_index):
        """
        Returns the indexes of the corners of a cell given its location xi,
        yi, zi

        Parameters
        ----------
        x_cell_index
        y_cell_index
        z_cell_index

        Returns
        -------

        """
        xcorner = np.array([0, 1, 0, 0, 1, 0, 1, 1])
        ycorner = np.array([0, 0, 1, 0, 0, 1, 1, 1])
        zcorner = np.array([0, 0, 0, 1, 1, 1, 0, 1])
        xcorners = x_cell_index[:, None] + xcorner[None, :]
        ycorners = y_cell_index[:, None] + ycorner[None, :]
        zcorners = z_cell_index[:, None] + zcorner[None, :]
        return xcorners, ycorners, zcorners
    
    def position_to_cell_corners(self,pos):
        inside = self.inside(pos)
        ix, iy, iz = self.position_to_cell_index(pos)
        cornersx, cornersy, cornersz = self.cell_corner_indexes(ix, iy, iz)
        globalidx = self.global_indicies(
            np.dstack([cornersx, cornersy, cornersz]).T)
        return globalidx, inside
    
    def position_to_cell_index(self,pos):
        ix = pos[:, 0] - self.origin[None, 0]
        iy = pos[:, 1] - self.origin[None, 1]
        iz = pos[:, 2] - self.origin[None, 2]
        ix = ix // self.step_vector[None, 0]
        iy = iy // self.step_vector[None, 1]
        iz = iz // self.step_vector[None, 2]
        return ix.astype(int), iy.astype(int), iz.astype(int)
    
    def node_indexes_to_position(self, xindex, yindex, zindex):

        x = self.origin[0] + self.step_vector[0] * xindex
        y = self.origin[1] + self.step_vector[1] * yindex
        z = self.origin[2] + self.step_vector[2] * zindex

        return np.array([x, y, z])
    
    def global_index_to_node_index(self, global_index):
        """
        Convert from global indexes to xi,yi,zi

        Parameters
        ----------
        global_index

        Returns
        -------

        """
        # determine the ijk indices for the global index.
        # remainder when dividing by nx = i
        # remained when dividing modulus of nx by ny is j

        x_index = global_index % self.nsteps[0, None]
        y_index = global_index // self.nsteps[0, None] % \
                  self.nsteps_cells[1, None]
        z_index = global_index // self.nsteps[0, None] // \
                  self.nsteps[1, None]
        return x_index, y_index, z_index

In [95]:
mesh = TetMesh([0,0,0],[10,10,10],[1.,1.,1.])
c = mesh.get_tetra([[.125,.125,.9],
                    [.9, .9, .9],
                    [.9, .9, .125],
                    [.5, .5, .75],
                    [.125, .9, .125],
                    [.125, .9, .125]
                   ])

In [96]:
c[0]

array([[[ 0.1  ,  0.125, -0.775,  1.55 ],
        [ 1.775,  0.1  , -1.75 ,  0.875],
        [ 0.875,  0.   , -0.775,  0.9  ],
        [ 0.875, -0.775,  0.   ,  0.9  ],
        [ 1.775,  0.125,  0.9  , -1.8  ]]])